In [11]:
# function to upload json file to S3 for staging
from s3 import upload_file
import requests as r
import json
from config import X_RAPIDAPI_KEY

### World Cup Team Extract

In [5]:
url_teams = "https://api-football-v1.p.rapidapi.com/v3/teams"

# world cup season 2022 
params = {
    "league":"1",
    "season":"2022"
}

headers = {
    "X-RapidAPI-Key": X_RAPIDAPI_KEY,
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

data = r.request("GET", url_teams, headers=headers, params=params).json()

upload_file(data, 'worldcup-project-2022-bucket', 'worldcup_teams_2022.json')

True

### Players in Each World Cup Squad Extract

In [5]:
# need to get an array of each world cup team id

    #  boto 3

import boto3

# establishes what AWS resource is being used = s3
s3 = boto3.resource('s3')
# Object class allows you to interact with the file or key in S3
obj = s3.Object('worldcup-project-2022-bucket', 'worldcup_teams_2022.json')
# get() retrieves S3 object meta data, but you can traverse into the ['Body'], which
# returns the StreamingBody()
data = obj.get()['Body'].read()

import json
# data came in as binary, json.loads turns into a dict type
data_json = json.loads(data)

team_ids = []
team_names = []

for x in data_json['response']:
    team_ids.append(x['team']['id'])

for y in data_json['response']:
    team_names.append(y['team']['name'])

print(team_ids)  
print(team_names)
# 2 missing teams: Australia, Costa Rica
# Australia ID: 20
# Costa Rice ID: 29

team_ids.append(20)
team_ids.append(29)
team_names.append('Australia')
team_names.append('Costa Rica')

[1, 2, 3, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 31, 767, 1118, 1504, 1530, 1569, 2382, 2384, 5529]
['Belgium', 'France', 'Croatia', 'Brazil', 'Uruguay', 'Spain', 'England', 'Japan', 'Senegal', 'Serbia', 'Switzerland', 'Mexico', 'South Korea', 'Denmark', 'Iran', 'Saudi Arabia', 'Poland', 'Germany', 'Argentina', 'Portugal', 'Tunisia', 'Morocco', 'Wales', 'Netherlands', 'Ghana', 'Cameroon', 'Qatar', 'Ecuador', 'USA', 'Canada']


In [14]:
# create a json file for each squad 

player_squad_url = "https://api-football-v1.p.rapidapi.com/v3/players/squads"

headers = {
	"X-RapidAPI-Key": X_RAPIDAPI_KEY,
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

for id in team_ids:
    querystring = {"team":id}
    data = r.request("GET", player_squad_url, headers=headers, params=querystring).json()
    upload_file(data, 'worldcup-project-2022-bucket', f'players_squad_{id}.json')

In [19]:
# test how to append json files to upload

seasons = [2022, 2021, 2020, 2019, 2018]

player_1 = {
    "id": 1,
    "name": "A"
}

player_2 = {
    "id": 2,
    "name": "B"
}

belgium_players = []
players = [player_1, player_2]
# for seasons in season:
#     for id in players:
#         belgium_players.append(id)

# from array of player ids in squad
for id in players:
    # seasons by static years
    for season in seasons:
        belgium_players.append(id)
        # get request
        # append to squad

print(belgium_players)

[{'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}, {'id': 2, 'name': 'B'}]
